In [1]:
import pandas as pd
import psycopg2
import scipy.sparse as sparse
import numpy as np
from scipy.sparse.linalg import spsolve
import sys
# import random
# import implicit
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics
import pickle
# from lightfm.data import Dataset
import csv
from cmfrec import CMF

# original_data = pd.read_csv('/home/wyf/Desktop/tmall/data.csv') # 在cold start里用
# original_data.drop(original_data.columns[0],axis=1,inplace=True)
# original_data['user_id'] = original_data.user_id.astype(int)

retail_data = pd.read_csv('/home/wyf/Desktop/tmall/train.csv')
# retail_data = pd.read_csv('train.csv')
retail_data.drop(retail_data.columns[0],axis=1,inplace=True)

retail_data['user_id'] = retail_data.user_id.astype(int)
data = retail_data[['user_id', 'item_id', 'Occurrence']] # Get rid of unnecessary info

item_lookup = retail_data[['item_id', 'cat_id']].drop_duplicates() # Only get unique item/description pairs
item_lookup['item_id'] = retail_data.item_id.astype(int) # Encode as strings for future lookup ease

grouped_cleaned = data.groupby(['user_id', 'item_id']).sum().reset_index() # Group together
grouped_cleaned.Occurrence.loc[grouped_cleaned.Occurrence == 0] = 1
grouped_purchased = grouped_cleaned.query('Occurrence > 0')

customers = list(np.sort(grouped_purchased.user_id.unique())) # Get our unique customers
products = list(grouped_purchased.item_id.unique()) # Get our unique products that were purchased
quantity = list(grouped_purchased.Occurrence)

retail_data.head()

/home/wyf/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/wyf/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/wyf/.local/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


,user_id,item_id,cat_id,seller_id,brand_id,time_stamp,action_type,age_range,gender,Occurrence
0,234512,179830,1208,546,2276,1111,2,5.0,0.0,1
1,217494,646334,1208,1304,2276,1111,2,4.0,1.0,2
2,278823,18090,1208,1304,2276,1111,2,6.0,1.0,4
3,271127,539608,1208,422,2276,1111,2,4.0,2.0,1
4,131687,717816,629,3912,2276,1010,2,4.0,0.0,1


In [2]:
temp = retail_data[retail_data['user_id'].isin(customers)]
user_info1 = temp[['user_id','age_range','gender']]
user_info = user_info1.drop_duplicates(subset=['user_id'])
# print(user_info)
new_user_info = pd.get_dummies(user_info, columns=['gender'])
user_info = pd.get_dummies(new_user_info, columns=['age_range'])
user_info.rename(columns={'user_id':'UserId'},inplace = True)
user_info.head()

,UserId,gender_0.0,gender_1.0,gender_2.0,age_range_0.0,age_range_1.0,age_range_2.0,age_range_3.0,age_range_4.0,age_range_5.0,age_range_6.0,age_range_7.0,age_range_8.0
0,234512,1,0,0,0,0,0,0,0,1,0,0,0
1,217494,0,1,0,0,0,0,0,1,0,0,0,0
2,278823,0,1,0,0,0,0,0,0,0,1,0,0
3,271127,0,0,1,0,0,0,0,1,0,0,0,0
4,131687,1,0,0,0,0,0,0,1,0,0,0,0


In [3]:
grouped_purchased.head()

,user_id,item_id,Occurrence
0,2,293244,1
1,2,917794,1
2,5,815566,1
3,14,267067,2
4,14,349999,2


In [4]:
with open ('/home/wyf/Desktop/tmall/test.txt', 'rb') as fp:
# with open ('test.txt', 'rb') as fp:
    test_set = pickle.load(fp) # 在explicit with cmf的变量名叫做product_users_altered
test_set[:5]

[(14783, 796443),
 (368853, 210741),
 (146302, 288118),
 (135570, 261644),
 (14094, 417530)]

In [5]:
retail_data.head()

,user_id,item_id,cat_id,seller_id,brand_id,time_stamp,action_type,age_range,gender,Occurrence
0,234512,179830,1208,546,2276,1111,2,5.0,0.0,1
1,217494,646334,1208,1304,2276,1111,2,4.0,1.0,2
2,278823,18090,1208,1304,2276,1111,2,6.0,1.0,4
3,271127,539608,1208,422,2276,1111,2,4.0,2.0,1
4,131687,717816,629,3912,2276,1010,2,4.0,0.0,1


In [6]:
grouped_purchased.columns = ['UserId','ItemId','Rating']
grouped_purchased.head()

,UserId,ItemId,Rating
0,2,293244,1
1,2,917794,1
2,5,815566,1
3,14,267067,2
4,14,349999,2


In [7]:
def pop_goods(data, num):
    
    '''找出买的最多的前num个商品'''
    data = data.copy()
    data['total_occ'] = data.groupby(['item_id']).cumcount(ascending=False)+1 # here every item appears only once to everybody
    data = data.drop_duplicates(subset=['item_id'], keep='first')
    goods = list(data.sort_values('total_occ', ascending=False).head(num).item_id)
    
    return goods
# popular = pop_goods(retail_data.copy(), 100)
# popular[:5]

In [8]:
'''
这个是比较好的参数
recommender = CMF(k=20, k_main=11, k_user=60, w_main=10.0, w_user=1.0, reg_param=1e-1, offsets_model=True,
                 add_user_bias=True, add_item_bias=False, reindex=True, random_seed=98, verbose=False)
# random_seed=98  50.5
'''

'\n这个是比较好的参数\nrecommender = CMF(k=20, k_main=11, k_user=60, w_main=10.0, w_user=1.0, reg_param=1e-1, offsets_model=True,\n                 add_user_bias=True, add_item_bias=False, reindex=True, random_seed=98, verbose=False)\n# random_seed=98  50.5\n'

In [9]:

recommender = CMF(k=20, k_main=0, k_user=60, w_main=10.0, w_user=1.0, reg_param=1e-1, offsets_model=True,
                 add_user_bias=True, add_item_bias=False, reindex=True, random_seed=98, verbose=False)
# random_seed=98  50.5

# recommender.fit(ratings=grouped_purchased.copy(), user_info=user_info.copy(),
#                 cols_bin_user=[cl for cl in user_info.columns if cl!='UserId'], cols_bin_item=None)
recommender.fit(ratings=grouped_purchased.copy(), user_info=user_info.copy(),
                cols_bin_user=None, cols_bin_item=None)

def accuracy(item, _list):
    if item in _list:
        return 1
    else:
        return 0

def acc_topN_with_cat(changed_pair, num, recom, users_information):
    altered_pair = changed_pair.copy()
    users_info = users_information.copy()
    count = 0
    popular = pop_goods(retail_data.copy(), 15)
#     print(altered_pair)
    for i in range(len(altered_pair)):
        '''以下是正常推荐(非冷启动)'''
        rec_item = list(recom.topN(user=altered_pair[i][0], n=300))
        rec_item = popular + rec_item
#         rec_item = pd.DataFrame(list(recom.topN(user=altered_pair[i][0], n=300)))
        rec_item = pd.DataFrame(rec_item)
#         print(len(rec_item))
#         print(altered_pair[i][0], rec_item[:5],'\n\n')
#         rec_item = pd.DataFrame(popular)
        rec_item.columns = ['item_id']
#         print(altered_pair[i][1])
#         print(rec_item)
#         print('user:', altered_pair[i][0])
#         print(rec_item,'\n')
#         print(altered_pair[i][0])
#         print(rec_item,'\n')
#         '''以下是冷启动'''
#         user = altered_pair[i][0]
#         a = users_info[users_info['UserId']==user].as_matrix()
# #         print(a,'\n')
#         user_attributes = np.delete(a, 0)
# #         print(user_attributes)
#         rec_item = list(recommender.topN_cold(attributes=user_attributes, n=100))


#         _item_cat = item_lookup[item_lookup['item_id'].isin(rec_item)]
        _item_cat = pd.merge(rec_item, item_lookup)
#         print(_item_cat)
#         print(_item_cat)
#         item_cat = _item_cat.drop_duplicates(subset=['item_id'])
#         print(item_cat)
        cat_topN = list(_item_cat.groupby('cat_id').head(num).item_id)
#         print(cat_topN[:5])
#         print(len(cat_topN))
        count = count + accuracy(altered_pair[i][1], cat_topN)

    return count/len(altered_pair)

topN_acc = acc_topN_with_cat(test_set, 5, recommender, user_info)
topN_acc

0.508695652173913